In [ ]:
# d = {'col1': [1, 2, 3, 4], 'col2': [5, 6, 7, 8]}
# df = dd.from_pandas(pd.DataFrame(data=d), npartitions=2)
# dd.to_parquet(df=df,
#               path='abfs://CONTAINER/FILE.parquet'
#               storage_options={'account_name': 'ACCOUNT_NAME',
#                                'account_key': 'ACCOUNT_KEY'}

In [ ]:
### QUEUED RUN - OLD CODE #### 

# def get_partitions_generator(
#     points_gdf: gpd.GeoDataFrame,
#     chunksize: int,
#     sort_points_by_hilbert_distance: bool = False,
# ) -> Generator[gpd.GeoDataFrame, None, None]:
#     """
#     Given a GeoDataFrame, this function creates a generator that returns chunksize
#     number of rows per iteration.

#     To be used for submitting Dask Futures.

#     Parameters
#     -----------
#     points_gdf : GeoDataFrame of points to be featurized.
#     chunksize : Number of points to be featurized per iteration.
#     sort_points_by_hilbert_distance : Whether to sort the points by their Hilbert distance.

#     Returns
#     --------
#     Generator
#     """

#     if sort_points_by_hilbert_distance:
#         points_gdf = _sort_points_by_hilbert_distance(points_gdf)

#     num_chunks = math.ceil(len(points_gdf) / chunksize)

#     logging.info(
#         f"Distributing {len(points_gdf)} points across {chunksize}-point partitions "
#         f"results in {num_chunks} partitions."
#     )

#     for i in range(num_chunks):
#         yield points_gdf.iloc[i * chunksize : (i + 1) * chunksize]


# def run_queued_futures_pipeline(
#     points_gdf: gpd.GeoDataFrame,
#     client: Client,
#     model: nn.Module,
#     satellite_name: str,
#     image_resolution: int,
#     image_dtype: str,
#     image_bands: list[str],
#     image_width: int,
#     min_image_edge: int,
#     sort_points_by_hilbert_distance: bool,
#     seasonal: bool,
#     year: int,
#     search_start: str,
#     search_end: str,
#     image_composite_method: str,
#     stac_api_name: str,
#     num_features: int,
#     device: str,
#     col_names: list,
#     n_concurrent_tasks: int,
#     chunksize: int,
#     output_folderpath: str,
# ) -> None:
#     """
#     For a given GeoDataFrame of coordinate points, this function partitions it
#     and submit each partition to be processed as a Future on the Dask client.

#     Initially, only as many partitions are submitted as there are threads. As each
#     partition is completed, another partition is submitted to the client.

#     Parameters
#     -----------
#     points_gdf : GeoDataFrame of points to be featurized.
#     client : Dask client.
#     model : PyTorch model to be used for featurization.
#     satellite_name : Name of the satellite to be used for featurization.
#     image_resolution : Resolution of the image to be generated.
#     image_dtype : Data type of the image to be generated.
#     image_bands : List of bands to be used for generating the image.
#     image_width : Desired width of the image to be fetched (in meters).
#     min_image_edge : Minimum edge length of the image to be generated.
#     sort_points_by_hilbert_distance : Whether to sort the points by their Hilbert distance.
#     seasonal : Whether to use seasonal imagery.
#     year : Year of imagery to be used.
#     search_start : Start date of imagery to be used.
#     search_end : End date of imagery to be used.
#     image_composite_method : Mosaic composite to be used.
#     stac_api_name : Name of the STAC API to be used.
#     num_features : Number of features to be extracted from the model.
#     device : Device to be used for featurization.
#     col_names : List of column names to be used for saving the features.
#     n_concurrent_tasks : Number of concurrent partitions to be submitted to the client.
#     chunksize : Number of points to be featurized per partition.
#     output_folderpath : Path to folder where features will be saved.

#     Returns
#     --------
#     None
#     """

#     # make generator for spliting up the data into partitions
#     partitions = get_partitions_generator(
#         points_gdf,
#         chunksize,
#         sort_points_by_hilbert_distance,
#     )

#     # if n_concurrent_tasks is not specified, use all available threads
#     if n_concurrent_tasks is None:
#         n_concurrent_tasks = sum(client.nthreads().values())
#     # if there are less partitions to run than concurrent tasks, run all partitions
#     n_concurrent_tasks = min(len(partitions), n_concurrent_tasks)

#     # kickoff "n_concurrent_tasks" number of tasks. Each of these will be replaced by a new
#     # task upon completion.
#     now = datetime.now().strftime("%d-%b %H:%M:%S")
#     logging.info(f"{now} Trying to kick off initial {n_concurrent_tasks} partitions...")
#     initial_futures_list = []
#     for i in range(n_concurrent_tasks):
#         try:
#             partition = next(partitions)
#         except StopIteration:
#             logging.info(
#                 f"There are less partitions than processors. All {i} partitions running."
#             )
#             wait(initial_futures_list)
#             break

#         future = client.submit(
#             run_pipeline,
#             points_gdf=partition,
#             model=model,
#             satellite_name=satellite_name,
#             image_resolution=image_resolution,
#             image_dtype=image_dtype,
#             image_bands=image_bands,
#             image_width=image_width,
#             min_image_edge=min_image_edge,
#             seasonal=seasonal,
#             year=year,
#             search_start=search_start,
#             search_end=search_end,
#             image_composite_method=image_composite_method,
#             stac_api_name=stac_api_name,
#             num_features=num_features,
#             device=device,
#             col_names=col_names,
#             output_folderpath=output_folderpath,
#             save_filename=f"df_{str(i).zfill(3)}.parquet.gzip",
#             return_df=False,
#         )
#         initial_futures_list.append(future)

#     # get generator that returns futures as they are completed
#     as_completed_generator = as_completed(initial_futures_list)

#     # only run each remaining partitions once a previous task has completed
#     for partition in partitions:
#         i += 1
#         completed_future = next(as_completed_generator)
#         logging.info(f"Adding partition {i}")

#         new_future = client.submit(
#             run_pipeline,
#             points_gdf=partition,
#             model=model,
#             satellite_name=satellite_name,
#             image_resolution=image_resolution,
#             image_dtype=image_dtype,
#             image_bands=image_bands,
#             image_width=image_width,
#             min_image_edge=min_image_edge,
#             seasonal=seasonal,
#             year=year,
#             search_start=search_start,
#             search_end=search_end,
#             image_composite_method=image_composite_method,
#             stac_api_name=stac_api_name,
#             num_features=num_features,
#             device=device,
#             col_names=col_names,
#             output_folderpath=output_folderpath,
#             save_filename=f"df_{str(i).zfill(3)}.parquet.gzip",
#             return_df=False,
#         )
#         as_completed_generator.add(new_future)

#     # wait for all futures to process
#     for completed_future in as_completed_generator:
#         pass

#     now = datetime.now().strftime("%d-%b %H:%M:%S")
#     logging.info(f"{now} Finished.")

In [ ]:
### BATCHED RUNNING. REQUIRES POINTS_GDF_WITH_STAC to start ####

# def run_partitions(
#     partitions: list,
#     satellite_config: dict,
#     featurization_config: dict,
#     model: nn.Module,
#     client: Client,
#     mosaiks_folder_path: str = None,
#     partition_ids: list = None,
# ) -> list:
#     """Run partitions in batches of n_per_run and save the result for each partition
#     to a parquet file. If a partition fails to be featurized, the partition ID is added
#     to a list and returned at the end of the run.

#     Parameters
#     ----------
#     partitions : List of dataframes.
#     satellite_config : Dictionary containing the satellite configuration.
#     featurization_config : Dictionary containing the featurization parameters.
#     model : PyTorch random convolutional feature model.
#     client : Dask client.
#     mosaiks_folder_path : Path to the folder where the mosaiks features should be saved.
#     partition_ids : List of partition IDs corresponding to each partition in `partitions`.
#         If None, the partition IDs will be inferred from the order of the
#         partitions in the list. Default is None.

#     Returns
#     -------
#     failed_ids : List of partition IDs that failed to be featurized.
#     """

#     n_per_run = featurization_config["dask"]["n_per_run"]
#     n_partitions = len(partitions)
#     logging.info(f"Running {n_partitions} partitions...")

#     if n_partitions < n_per_run:
#         logging.info(
#             f"n_partitions is smaller than n_per_run. Running all {n_partitions} partitions."
#         )
#         n_per_run = n_partitions

#     if partition_ids is None:
#         partition_ids = list(range(n_partitions))

#     mosaiks_column_names = [
#         f"mosaiks_{i}" for i in range(featurization_config["model"]["num_features"])
#     ]

#     failed_ids = []
#     checkpoint_indices = np.arange(0, n_partitions + n_per_run, n_per_run)
#     for p_start_id, p_end_id in zip(checkpoint_indices[:-1], checkpoint_indices[1:]):

#         now = datetime.now().strftime("%d-%b %H:%M:%S")
#         logging.info(f"{now} Running batch: {p_start_id} to {p_end_id - 1}")

#         batch_indices = list(range(p_start_id, p_end_id))
#         batch_p_ids = [partition_ids[i] for i in batch_indices]
#         batch_partitions = [partitions[i] for i in batch_indices]

#         failed_ids += run_batch(
#             partitions=batch_partitions,
#             partition_ids=batch_p_ids,
#             satellite_config=satellite_config,
#             featurization_config=featurization_config,
#             mosaiks_column_names=mosaiks_column_names,
#             model=model,
#             client=client,
#             mosaiks_folder_path=mosaiks_folder_path,
#         )

#     return failed_ids


# def run_batch(
#     partitions: list,
#     partition_ids: list,
#     satellite_config: dict,
#     featurization_config: dict,
#     mosaiks_column_names: list,
#     model: nn.Module,
#     client: Client,
#     mosaiks_folder_path: str,
# ) -> list:
#     """
#     Run a batch of partitions and save the result for each partition to a parquet file.

#     Parameters
#     ----------
#     partitions :List of dataframes to process.
#     partition_ids : List containing IDs corresponding to the partitions passed (to be used
#         for naming saved files and reference in case of failure).
#     satellite_config : Dictionary containing the satellite configuration.
#     featurization_config : Dictionary containing the featurization parameters.
#     model : PyTorch random convolutional feature model.
#     client : Dask client.
#     mosaiks_folder_path : Path to the folder where the mosaiks features should be saved.

#     Returns
#     -------
#     failed_ids : List of partition labels that failed to be featurized.
#     """

#     failed_ids = []
#     delayed_dfs = []

#     # collect futures
#     for p_id, p in zip(partition_ids, partitions):
#         str_id = str(p_id).zfill(3)  # makes 1 into '001'

#         f = delayed_partition_run(
#             df=p,
#             satellite_config=satellite_config,
#             featurization_config=featurization_config,
#             mosaiks_column_names=mosaiks_column_names,
#             model=model,
#             dask_key_name=f"features_{str_id}",
#         )
#         delayed_dfs.append(f)

#     # delayed -> futures -> collected results
#     futures_dfs = client.compute(delayed_dfs)
#     failed_ids = collect_results(
#         futures_dfs=futures_dfs, mosaiks_folder_path=mosaiks_folder_path
#     )

#     # prep for next run
#     client.restart()
#     sleep(5)

#     return failed_ids


# def collect_results(futures_dfs: list, mosaiks_folder_path: str) -> list:
#     """
#     Save computed dataframes to parquet files. If a partition fails to be featurized,
#     the partition ID is added to a list.

#     Parameters
#     ----------
#     futures_dfs : List of futures containing the computed dataframes.
#     mosaiks_folder_path : Path to the folder where the mosaiks features should be saved.

#     Returns
#     -------
#     failed_ids : List of partition IDs that failed to be featurized.
#     """

#     failed_ids = []
#     for f in as_completed(futures_dfs):
#         try:
#             df = f.result()
#             utl.save_dataframe(
#                 df=df, file_path=f"{mosaiks_folder_path}/df_{f.key}.parquet.gzip"
#             )
#         except Exception as e:
#             f_key = f.key
#             partition_id = int(f_key.split("features_")[1])
#             logging.info(f"Partition {partition_id} failed. Error:", e)
#             failed_ids.append(partition_id)

#     return failed_ids


# def run_single_partition(
#     partition: pd.DataFrame,
#     satellite_config: dict,
#     featurization_config: dict,
#     model: nn.Module,
#     client: Client,
# ) -> pd.DataFrame:
#     """Run featurization for a single partition. For testing.

#     Parameters
#     ----------
#     partition : Dataframe containing the data to featurize.
#     satellite_config : Dictionary containing the satellite configuration.
#     featurization_config : Dictionary containing the featurization parameters.
#     model : PyTorch random convolutional feature model.
#     client : Dask client.

#     Returns
#     -------
#     df : Dataframe containing the featurized data.
#     """

#     mosaiks_column_names = [
#         f"mosaiks_{i}" for i in range(featurization_config["model"]["num_features"])
#     ]

#     f = delayed_partition_run(
#         df=partition,
#         satellite_config=satellite_config,
#         featurization_config=featurization_config,
#         mosaiks_column_names=mosaiks_column_names,
#         model=model,
#         dask_key_name="single_run",
#     )

#     df_future = client.compute(f)
#     for f in as_completed([df_future]):
#         df = f.result()

#     return df


# @delayed
# def delayed_partition_run(
#     df: pd.DataFrame,
#     satellite_config: dict,
#     featurization_config: dict,
#     mosaiks_column_names: list,
#     model: nn.Module,
# ) -> pd.DataFrame:
#     """Run featurization for a single partition."""

#     data_loader = create_data_loader(
#         points_gdf_with_stac=df,
#         satellite_params=satellite_config,
#         batch_size=featurization_config["model"]["batch_size"],
#     )

#     X_features = create_features(
#         dataloader=data_loader,
#         n_features=featurization_config["model"]["num_features"],
#         n_points=len(df),
#         model=model,
#         device=featurization_config["model"]["device"],
#         min_image_edge=satellite_config["min_image_edge"],
#     )

#     df = pd.DataFrame(
#         data=X_features, index=df.index.copy(), columns=mosaiks_column_names
#     )

#     return df
